In [ ]:
"""
Practice and explore networkx with speaker_word [topic_word]

nodes: all speakers + all words (after cleaning, no stop words/boring words, etc)

edges: 

"""

In [1]:
import re, itertools
import networkx as nx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from collections import Counter

from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud

from gensim import matutils, models
import scipy.sparse

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_4/py')
from word_cloud import *
import importlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
topic_df = pd.read_pickle('../dump/topic_df')
speech_df = pd.read_pickle('../dump/speech_clean_lemma')
topic_word = pd.read_pickle('../dump/to_nx_topic_words')

In [3]:
speech_df

,speaker,year,transcript,length
0,SIDDHARTHA MUKHERJEE,2018,i wish someone have tell me at my own commence...,14487
1,ABBY WAMBACK,2018,failure be not something to be ashamed of its ...,15866
2,JON B. FISHER,2018,thank you very much my father commencement spe...,8544
3,MINDY KALING,2018,good morning to the class of the faculty the p...,15391
4,JESMYN WARD,2018,persist be patient be well good morning it be ...,14063
...,...,...,...,...
436,CARRIE CHAPMAN,1936,i bring a message to sweet briar college and e...,22942
437,FRANKLIN D ROOSEVELT,1932,for me as for you this be a day of honorable a...,16017
438,OPRAH WINFREY,1918,thank you wallis annenberg and a special thank...,15301
439,RALPH WALDO,1838,in this refulgent summer it have be a luxury t...,40403


In [4]:
topic_word

,Topic #01,Topic #02,Topic #03,Topic #04,advice,career,company,country,course,dream,...,place,president,question,sense,state,success,university,women,word,year
Topic #01,0,0,0,0,2,0,0,0,0,2,...,2,0,0,0,0,0,0,0,0,2
Topic #02,0,0,0,0,0,0,0,2,0,0,...,2,2,0,2,0,0,0,0,2,2
Topic #03,0,0,0,0,0,0,2,0,2,2,...,0,0,2,0,2,0,2,2,0,2
Topic #04,0,0,0,0,0,2,0,2,2,0,...,2,0,0,0,0,2,2,2,2,0
advice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
career,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
company,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
country,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
course,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dream,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import pickle

with open("../dump/common_words.txt", "rb") as f:   # Unpickling
    common_words = pickle.load(f)

In [11]:
# Bag of words with CountVectorizer
# add_stop_words selected from after lemmatization
# will also remove common_words (most commonly used words in all speeches)
# will also remove boring words (words that do not add much insight to topic modeling)
add_stop_words = ['like','youre','ive','im','really','id','ve','just','dont','thi','wa',
                  'say','know','make','people']

boring_words = ['say','like','just','dont','don','im',
                  'ive','youll','youve','things','thing','youre','right','really','lot',
                  'make','know','people','way','day','class']


add_stop_words = add_stop_words + common_words + boring_words

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(speech_df.transcript)

In [12]:
# If add_stop_words is modified, update tdm
data_dtm = cv_dtm(speech_df,'transcript',add_stop_words)
tdm = data_dtm.transpose()

In [14]:
data_dtm

,aa,aahhhh,aaron,aback,abalthus,abandon,abandonment,abate,abbot,abbreviation,...,ômay,ôsobriety,ôtell,ôthe,ôwe,ôwhat,ôyou,ôyouõre,über,我是一只迷途的羔羊谁能引我走出迷茫哪有我重生的希望噢牧羊人啊噢牧羊人啊
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
speech_df.transcript

0      i wish someone have tell me at my own commence...
1      failure be not something to be ashamed of its ...
2      thank you very much my father commencement spe...
3      good morning to the class of the faculty the p...
4      persist be patient be well good morning it be ...
                             ...                        
436    i bring a message to sweet briar college and e...
437    for me as for you this be a day of honorable a...
438    thank you wallis annenberg and a special thank...
439    in this refulgent summer it have be a luxury t...
440    its an honor to be with so many smart people a...
Name: transcript, Length: 441, dtype: object

In [39]:
def corpus_remove_stop_words(transcript):
    
    add_stop_words = ['like','youre','ive','im','really','id','ve','just','dont','thi','wa',
                  'say','know','make','people']

    boring_words = ['say','like','just','dont','don','im',
                      'ive','youll','youve','things','thing','youre','right','really','lot',
                      'make','know','people','way','day','class']


    add_stop_words = add_stop_words + common_words + boring_words

    stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

    words = transcript.split(" ")
    new_words = [x for x in words if x not in stop_words]
    new_transcript = " ".join(new_words)
    return new_transcript

In [27]:
len(corpus_remove_stop_words(speech_df.transcript.iloc[0])),len(speech_df.transcript.iloc[0])

(6829, 13441)

In [35]:
test_df

,transcript
SIDDHARTHA MUKHERJEE,i wish someone have tell me at my own commence...
ABBY WAMBACK,failure be not something to be ashamed of its ...
JON B. FISHER,thank you very much my father commencement spe...
MINDY KALING,good morning to the class of the faculty the p...
JESMYN WARD,persist be patient be well good morning it be ...


In [40]:
test_df.transcript.apply(corpus_remove_stop_words)

SIDDHARTHA MUKHERJEE    wish commencement requirement graduation emerg...
ABBY WAMBACK            failure ashamed power failure highest octane f...
JON B. FISHER           thank father commencement speaker martin luthe...
MINDY KALING            morning faculty parent grandparents fellow hon...
JESMYN WARD             persist patient morning honor pleasure share m...
Name: transcript, dtype: object

In [43]:
pd.DataFrame(test_df.transcript.apply(corpus_remove_stop_words)).to_pickle('../dump2/test_df')

In [44]:
pd.DataFrame(test_df.transcript.apply(corpus_remove_stop_words)).to_csv('../dump2/test_df.csv')

In [53]:
speech_df.iloc[:50].to_csv('../dump2/test_df.csv')

In [51]:
speech_df.iloc[:50].speaker.nunique()

48